# Environment Setup
## City Overview
Simulate a virtual city with a population of 500,000. The city is divided into 12 administration districts, each containing a number of sub-regions (cells). These sub-regions are randomly assigned as work locations, education facilities, social places, and residential areas.

## City Structure
* Total Cells: 1,000,000.
* Total Population: 500,000.
* Number of Administrative Districts: 12

## Location Distribution
Within the 1,000,000 cells, the distribution of different types of locations:
* Work Locations: Some cells will be randomly designated as work locations.
* Education Facilities: Some cells will be randomly designated as schools.
* Social Places: Additional cells will be randomly designated as social places, such as parks, cafes, shopping malls, etc.
* Residential Areas: The remaining cells will be designated as residential areas.

## Daily Time Point
The simulation will include three key time points each dat:
* Morning: Agents travel to work, school, or other daily activities
* Evening: Agents return home or visit social places
* Night: Agents stay at home resting

In [1]:
pip install mesa

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.space import NetworkGrid, MultiGrid
from mesa.time import RandomActivation

import networkx as nx
from datetime import datetime, timedelta
from mesa.datacollection import DataCollector

# Virus Description
The spread of the virus within the simulation is defined by several parameters, such as:
* Transmission Rate
* Latency Period
* Infection Duration: Recovery Rate, Death Rate
* ......

# Agent Description
## Agent Attributes
Each agent in the simulation will have the following attributes:

### 1. Age:
* Represents the age of the agent, affecting their behavior, occupation and risk of severe disease.

### 2. Occupation:
Defines the agents's primary daily activity and location. Occupations include student, worker, retiree, etc.

### 3. Health Status (SEIRD):
* Susceptible (S): The agent is healthy but can become infected
* Exposed (E): The agent has been exposed to the virus, but it is not yet infectious
* Infected (I): The agent is infected and can spread the virus
* Recovered (R): The agent has recovered from the infection and is immune
* Deceased (D): The agent has died from the infection.

### 4. Mask Wearing Status:
* Whether the agent is currently wearing a mask (yes/no)

### 5. Compilance Level with Lockdown Policies:
* Measures how strictly the agent follows lockdown and social distancing guidelines (as probabilities)

## Memory
Agents have the capability to remeber certain types of information:
* Health Status Changes:
  Agents can remeber their past health status, that interferes their choices for making traveling decisions, mask-wearing and compilance with lockdown policies
* Social Interactions:  
  Agents can record their social interactions, close contacts may lead to infection
* Environmental Information:
  Agents can retain information about the environment, such as the severity of outbreaks in specific areas, helping them make decisions about their movements and interacitons.

## Learning
Agents can adapt their behavior based on past experience and acquired knowledge:
* Behavioral Adjustment:
* Risk Adjustment:

## Agent Behavior Descriptions
### 1. Daily Activities:
    Agents follow a daily routine that involves going to work, school, or engaging in social activities. These routine vary depending on the agent's occupation and age group. The nature of activities change throughout different times of the day, include morning, evening and night.

### 2. Social Interactions:
    Agents engage in face-to-face interactions with other agents throughout the day. These interactions occur at work, school, social places and within residential areas.

### 3. Mask-Wearing Behavior:
    Agents decide whether to wear masks based on their health status, perceived risk, and compliance level.

### 4. Lockdown Policies:
    Agents' behavior is influenced by lockdown policies in place, such as movement restrictions and lockdowns.

In [3]:
health_stats = ["S", "E", "I", "R", "D"]
self.health_stats = health_stats
self.mask = True 

NameError: name 'self' is not defined

## SIERD Model
* Susceptible: These agents have not yet been infected with the virus but are at risk of becoming infected  
* Exposed: These agents have been exposed to the virus and are infected, but are not yet infectious to others  
* Infected: These agents are currently infected and are capable of transmitting the virus to susceptible individuals    
* Recovered: These agents have recovered from the infection and are typically assumed to have immunity    
* Dead: These agents have died as a result of the infection

## Rules
### 1. Infection Spread Rules:
* When an infected agent comes into contact with a susceptible agent, there is a probability of virus transmission
* The probability of transmission may increase when a susceptible agent comes into contact with multiple infected agents
### 2. Social Interaction Rules:
* Agents’ social behavior may be influenced by their infection status and the surrounding environment. For example, during outbreaks, agents may be  more inclined to avoid crowded areas or maintain social distancing.
### 3. Government Intervention Rules:
* Government may implement control measures such as lockdowns, mask-wearing policies travel to slow down virus spread.

# Government Policies
* No Interventions
* Lockdown Only
* Mask Policy Only
* Combination of Lockdown and Mask Policy

In [ ]:
import random
import matplotlib.pyplot as plt
import networkx as nx
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid

class CityAgent(Agent):
    def __init__(self, unique_id, model, residence, work):
        super().__init__(unique_id, model)
        self.residence = residence
        self.work = work
        self.location = residence

    def move_to_work(self):
        self.location = self.work

    def move_to_residence(self):
        self.location = self.residence

    def step(self):
        # Assume the day has two time periods: morning and afternoon
        if self.model.time_of_day == "morning":
            self.move_to_work()
        elif self.model.time_of_day == "afternoon":
            self.move_to_residence()

class CityModel(Model):
    def __init__(self, num_agents, num_districts):
        self.num_agents = num_agents
        self.num_districts = num_districts
        self.schedule = RandomActivation(self)
        self.grid = NetworkGrid(nx.Graph())
        self.time_of_day = "morning"
        
        self.districts = self.create_districts()
        self.add_nodes_to_grid()

        # Create agents
        for i in range(self.num_agents):
            residence = random.choice(list(self.districts.keys()))
            work = random.choice(list(self.districts.keys()))
            agent = CityAgent(i, self, residence, work)
            self.schedule.add(agent)
            agent_node = self.grid.G.nodes[residence]
            agent_node["agent"] = agent  # Set the agent attribute for the node

    def create_districts(self):
        districts = {}
        for i in range(self.num_districts):
            residence_area = i * 2  # ensure unique IDs
            work_area = i * 2 + 1   # ensure unique IDs
            districts[residence_area] = {"type": "residence", "agents": []}
            districts[work_area] = {"type": "work", "agents": []}
        return districts

    def add_nodes_to_grid(self):
        for district in self.districts:
            self.grid.G.add_node(district)

    def step(self):
        self.schedule.step()
        self.time_of_day = "afternoon" if self.time_of_day == "morning" else "morning"

# 設定模型參數
num_agents = 100
num_districts = 12

# 初始化模型
model = CityModel(num_agents, num_districts)

# 模擬步驟
for i in range(10):
    model.step()

# 可視化
def plot_agents(model):
    pos = nx.spring_layout(model.grid.G)  # use spring layout for better visualization
    plt.figure(figsize=(15, 15))
    nx.draw(model.grid.G, pos, with_labels=True, node_size=500, node_color="skyblue", font_size=10, font_color="black")

    for agent in model.schedule.agents:
        x, y = pos[agent.location]
        plt.scatter(x, y, c="red" if agent.location == agent.work else "green", s=100)

    plt.show()

plot_agents(model)


## Setup environment
small demo

In [ ]:
import random
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid

class CityAgent(Agent):
    def __init__(self, unique_id, model, residence_area, work_area):
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.residence_area = residence_area
        self.work_area = work_area
        self.location = residence_area
        self.model = model

    def move_to_work(self):
        self.model.grid.move_agent(self, self.work_area)
        self.location = self.work_area

    def move_to_residence(self):
        self.model.grid.move_agent(self, self.residence_area)
        self.location = self.residence_area

    def step(self):
        if self.model.time_of_day == "morning":
            self.move_to_work()
        elif self.model.time_of_day == "afternoon":
            self.move_to_residence()

class CityModel(Model):
    def __init__(self, num_agents, width, height):
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.time_of_day = "morning"
        
        # Create agents
        for i in range(self.num_agents):
            residence_area = (random.randint(0, width - 1), random.randint(0, height - 1))
            work_area = (random.randint(0, width - 1), random.randint(0, height - 1))
            agent = CityAgent(i, self, residence_area, work_area)
            self.schedule.add(agent)
            self.grid.place_agent(agent, residence_area)

    def step(self):
        self.schedule.step()
        self.time_of_day = "afternoon" if self.time_of_day == "morning" else "morning"

# Set up the model
num_agents = 15
width = 8
height = 8

model = CityModel(num_agents, width, height)

# Simulate for a few steps
for i in range(10):  # Simulate 10 steps
    print(f"Step {i+1}")
    if model.time_of_day == "morning":
        print("Morning:")
    else:
        print("Afternoon:")
    model.step()
    for agent in model.schedule.agents:
        print(f"Agent {agent.unique_id} is at {agent.location}")
    print("\n")

# Visualize the model
def plot_model(model):
    colors = ["blue", "green", "red", "yellow", "purple", "orange", "pink", "brown"]
    plt.figure(figsize=(8, 8))
    for cell in model.grid.coord_iter():
        content, (x, y) = cell
        for agent in content:
            plt.scatter(x, y, color=colors[agent.unique_id % len(colors)], s=100)  # Adjust color based on unique_id
    plt.show()

plot_model(model)

### Define CityAgent
* unique_id: A unique identifier for each agent.
* residence_area: The area where the agent resides.
* work_area: The area where the agent works.

#### Agent movement rules:

* If the agent is not in a lockdown area, they can travel to their workplace in the morning and return home in the afternoon.
* If the agent's residence area is in lockdown, they cannot move.
* If the agent's workplace is in lockdown, they stay at home.

#### Daily routine:

* Every morning, agents travel to their workplace.
* Every afternoon, agents return home.

In [ ]:
class CityAgent(Agent):
    def __init__(self, unique_id, model, residence_area, work_area, district):
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.residence_area = residence_area
        self.work_area = work_area
        self.district = district
        self.location = residence_area
        self.model = model

    def move_to_work(self):
        if not self.model.is_lockdown(self.work_area):
            self.model.grid.move_agent(self, self.work_area)
            self.location = self.work_area

    def move_to_residence(self):
        if not self.model.is_lockdown(self.residence_area):
            self.model.grid.move_agent(self, self.residence_area)
            self.location = self.residence_area

    def step(self):
        if self.model.time_of_day == "morning":
            self.move_to_work()
        elif self.model.time_of_day == "afternoon":
            self.move_to_residence()


### Define City
1. A model representing a city with agents that have a residence area and a work area.
2. Agents can move between these areas based on the time of day and lockdown status of districts.


In [ ]:
class CityModel(Model):
    ''' 
        Attributes:
        -----------
        num_agents : int
            The number of agents in the model.
        schedule : mesa.time.RandomActivation
            The scheduler to manage agent activation.
        grid : mesa.space.MultiGrid
            The grid representing the city space.
        time_of_day : str
            Represents the current time of day ('morning' or 'afternoon').
        lockdown_areas : set
            A set of district IDs that are under lockdown.
        lockdown_coordinates : dict
            A dictionary mapping district IDs to their coordinates that are under lockdown.
        districts : dict
            A dictionary mapping grid coordinates to district IDs
        
        Methods:
        --------
        __init__(self, num_agents, width, height, num_districts):
            Initializes the CityModel with the given number of agents, grid size, and number of districts.
        create_districts(self, num_districts, width, height):
            Creates districts by dividing the grid into specified number of districts and returns a dictionary mapping coordinates to district IDs.
        is_lockdown(self, area):
            Checks if the given area (grid coordinate) is in a lockdown district.
        set_lockdown(self, district_id):
            Sets a district to be in lockdown and stores its coordinates.
        lift_lockdown(self, district_id):
            Lifts the lockdown for a district and removes its coordinates from the lockdown dictionary.
        print_lockdown_areas(self):
            Prints the coordinates of all districts that are currently under lockdown.
        print_districts(self):
            Prints the coordinates of all districts in the model.
        step(self):
            Advances the model by one step, switching between 'morning' and 'afternoon'.
    
     '''
    
    def __init__(self, num_agents, width, height, num_districts):
        """
        Initializes the CityModel with the given number of agents, grid size, and number of districts.

        Parameters:
        -----------
        num_agents : int
            The number of agents in the model.
        width : int
            The width of the grid.
        height : int
            The height of the grid.
        num_districts : int
            The number of districts in the city.
        """     
     
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.time_of_day = "morning"
        self.lockdown_areas = set()
        self.lockdown_coordinates = {}

        self.districts = self.create_districts(num_districts, width, height)
        
        # Create agents in the environment 
        for i in range(self.num_agents):
            residence_area = (random.randint(0, width - 1), random.randint(0, height - 1))
            work_area = (random.randint(0, width - 1), random.randint(0, height - 1))
            # all agents start at residence area
            district = self.districts[residence_area]
            agent = CityAgent(i, self, residence_area, work_area, district)
            self.schedule.add(agent)
            self.grid.place_agent(agent, residence_area)
        
        # Record the agents' district movements
        self.agent_movements = {i: [] for i in range(self.num_agents)}

    def create_districts(self, num_districts, width, height):
        """
        Creates districts by dividing the grid into specified number of districts.

        Parameters:
        -----------
        num_districts : int
            The number of districts in the city.
        width : int
            The width of the grid.
        height : int
            The height of the grid.

        Returns:
        --------
        dict
            A dictionary mapping coordinates to district IDs.
        """
        districts = {}
        num_cells = width * height
        cells_per_district = num_cells // num_districts
        district_id = 0
        for x in range(width):
            for y in range(height):
                if len([v for v in districts.values() if v == district_id]) >= cells_per_district:
                    district_id += 1
                if district_id >= num_districts:
                    district_id = num_districts - 1
                districts[(x, y)] = district_id
        return districts

    def is_lockdown(self, area):
        # Checks if the given area is in a lockdown area
        return self.districts.get(area) in self.lockdown_areas

    def set_lockdown(self, district_id):
        # Assign a district to be in lockdown and store its coordinates
        self.lockdown_areas.add(district_id)
        self.lockdown_coordinates[district_id] = [coord for coord, dist in self.districts.items() if dist == district_id]

    def lift_lockdown(self, district_id):
        # Lift the lockdown for a district and removes its coordinates from the lockdown disctionary
        self.lockdown_areas.discard(district_id)
        if district_id in self.lockdown_coordinates:
            del self.lockdown_coordinates[district_id]

    def print_lockdown_areas(self):
        # Print the coordinates of all the districts that are under lockdown
        for district_id, coords in self.lockdown_coordinates.items():
            print(f"Lockdown District {district_id}: {coords}")

    def print_districts(self):
        # Print all the coordinates of all districts in the model
        district_coords = {}
        for coord, district_id in self.districts.items():
            if district_id not in district_coords:
                district_coords[district_id] = []
            district_coords[district_id].append(coord)
        
        for district_id, coords in district_coords.items():
            print(f"District {district_id}: {coords}")

    def step(self):
        # Advance the model by one step, switching between "morning" and "afternoon"
        self.schedule.step()
        # Record agents' district for the current timestep
        for agent in self.schedule.agents:
            self.agent_movements[agent.unique_id].append(self.districts[agent.location])
        self.time_of_day = "afternoon" if self.time_of_day == "morning" else "morning"

### Test Here!
try different
* num_agents : number of agents
* width: width of the environment
* height: height of the environment
* num_districts: number of districts in the environment

In [ ]:
# Set up the model
num_agents = 15
width = 8
height = 8
num_districts = 10

model = CityModel(num_agents, width, height, num_districts)

In [ ]:
# Example of setting lockdown on a district
model.set_lockdown(3)  # Lockdown the (X) district

# Print lockdown districts
model.print_lockdown_areas()

# Print all districts and their coordinates
model.print_districts()

In [ ]:
# Simulate for a few steps
num_steps = 10
for i in range(num_steps):  # Simulate 10 steps
    print(f"Step {i+1}")
    if model.time_of_day == "morning":
        print("Morning:")
    else:
        print("Afternoon:")
    model.step()
    for agent in model.schedule.agents:
        print(f"Agent {agent.unique_id} is at {agent.location} in district {agent.district}")
    print("\n")

# Visualize the model
def plot_agent_movements(model):
    plt.figure(figsize=(12, 6))
    for agent_id, movements in model.agent_movements.items():
        plt.plot(range(len(movements)), movements, label=f"Agent {agent_id}")
    plt.xlabel("Timestep")
    plt.xticks(range(num_steps))  # Ensure all timesteps are shown
    plt.ylabel("District")
    plt.yticks(sorted(set(sum(model.agent_movements.values(), []))))  # Ensure all districts are shown
    plt.title("Agent Movements Across Districts Over Time")
    plt.legend()
    plt.show()

plot_agent_movements(model)

#### SEIRD model combine with the rules

In [ ]:
class SIERDAgent(Agent):
    def __init__(self, unique_id, model, wearing_mask=False, isolated=False, recovered=False):
        super().__init__(unique_id, model)
        self.state = "Susceptible"  # Initial state
        self.infection_time = 0
        self.wearing_mask = wearing_mask
        self.isolated = isolated
        self.recovered = recovered

    def step(self):
        if self.state == "Susceptible":
            self.check_exposure()
        elif self.state == "Exposed":
            self.progress_to_infected()
        elif self.state == "Infected":
            self.infect_others()
            self.progress_to_recovered_or_dead()
        elif self.state == "Recovered":
            self.check_reinfection()
        elif self.state == "Dead":
            pass  # No action

    def check_exposure(self):
        if self.isolated:
            return
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        for neighbor in neighbors:
            if neighbor.state == "Infected":
                transmission_rate = self.model.transmission_rate
                if self.wearing_mask:
                    transmission_rate *= 0.2
                if random.random() < transmission_rate:
                    self.state = "Exposed"
                    self.infection_time = self.model.schedule.time
                    break

    def progress_to_infected(self):
        if self.model.schedule.time - self.infection_time >= self.model.latency_period:
            self.state = "Infected"

    def infect_others(self):
        if self.isolated:
            return
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        for neighbor in neighbors:
            if neighbor.state == "Susceptible":
                transmission_rate = self.model.transmission_rate
                if self.wearing_mask:
                    transmission_rate *= 0.2
                if neighbor.recovered:
                    transmission_rate *= 0.5  # Lower rate for those who have recovered
                if random.random() < transmission_rate:
                    neighbor.state = "Exposed"
                    neighbor.infection_time = self.model.schedule.time

    def progress_to_recovered_or_dead(self):
        if self.model.schedule.time - self.infection_time >= self.model.infection_duration:
            if random.random() < self.model.recovery_rate:
                self.state = "Recovered"
                self.recovered = True
                if not self.wearing_mask:  # Recovered individuals are more likely to wear masks
                    self.wearing_mask = random.random() < 0.8
            else:
                self.state = "Dead"

    def check_reinfection(self):
        if self.isolated:
            return
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        for neighbor in neighbors:
            if neighbor.state == "Infected":
                transmission_rate = self.model.transmission_rate  # Same reinfection rate as initial infection
                if self.wearing_mask:
                    transmission_rate *= 0.2
                if random.random() < transmission_rate:
                    self.state = "Exposed"
                    self.infection_time = self.model.schedule.time
                    break









In [ ]:
class SIERDModel(Model):
    def __init__(self, width, height, density, transmission_rate, latency_period, infection_duration, recovery_rate, policy):
        self.num_agents = int(width * height * density)
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.transmission_rate = transmission_rate
        self.latency_period = latency_period
        self.infection_duration = infection_duration
        self.recovery_rate = recovery_rate
        self.policy = policy

        for i in range(self.num_agents):
            wearing_mask = False
            isolated = False
            if policy == "Mask Policy Only":
                wearing_mask = True
            elif policy == "Lockdown Only":
                isolated = True
            elif policy == "Combination of Lockdown and Mask Policy":
                wearing_mask = True
                isolated = True
            
            agent = SIERDAgent(i, self, wearing_mask, isolated)
            self.schedule.add(agent)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

        # Initialize some agents as infected
        infected_agents = self.random.sample(self.schedule.agents, initial_infected)
        for agent in infected_agents:
            agent.state = "Infected"
            agent.infection_time = self.schedule.time

        self.datacollector = DataCollector(
            agent_reporters={"State": "state"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
# Parameters
width = 50
height = 50
density = 0.1
transmission_rate = 0.2
latency_period = 3
infection_duration = 14
recovery_rate = 0.95
initial_infected = 10 

# Define policies
policies = ["No Interventions", "Lockdown Only", "Mask Policy Only", "Combination of Lockdown and Mask Policy"]

# Run model for each policy
for policy in policies:
    print(f"Running model with policy: {policy}")
    model = SIERDModel(width, height, density, transmission_rate, latency_period, infection_duration, recovery_rate, policy)
    for i in range(100):
        model.step()
    
    # Data analysis
    data = model.datacollector.get_agent_vars_dataframe()
    print(data)

In [ ]:
# 加入Agent behaviour

In [4]:
# CityAgent 类定义
class CityAgent(Agent):
    def __init__(self, unique_id, model, residence, work, school, compliance_level):
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.residence_area = residence_area
        self.work_area = work_area
        self.school_area = school_area
        self.district = district
        self.location = residence_area
        self.model = model
        self.health_status = "Healthy"  
        self.wearing_mask = False  
        self.isolated = False  
        self.went_to_social = False  

    def move_to_work(self):
        if not self.model.is_lockdown(self.work_area):
            self.model.grid.move_agent(self, self.work_area)
            self.location = self.work_area

    def move_to_residence(self):
        if not self.model.is_lockdown(self.residence_area):
            self.model.grid.move_agent(self, self.residence_area)
            self.location = self.residence_area

    def move_to_school(self):
        school_area = random.choice(list(self.model.districts.keys()))
        if not self.model.is_lockdown(school_area):
            self.model.grid.move_agent(self, school_area)
            self.location = school_area

    def perform_social_activities(self):
        # move to random social places
        social_area = random.choice(self.model.social_places)
        if not self.model.is_lockdown(social_area):
            self.model.grid.move_agent(self, social_area)
            self.location = social_area
            self.went_to_social = True

    def decide_mask_wearing(self):
        if self.health_status == "Infected" or random.random() < self.model.perceived_risk:
            self.wearing_mask = True
        else:
            self.wearing_mask = False

    def step(self):
        if self.model.time_of_day == "morning":
            activity_choice = random.choice(["work", "school", "residence"])
            if activity_choice == "work":
                self.move_to_work()
            elif activity_choice == "school":
                self.move_to_school()
            else:
                self.move_to_residence()
        elif self.model.time_of_day == "afternoon":
            if random.random() < 0.5:
                self.perform_social_activities()
            else:
                self.move_to_residence()
        elif self.model.time_of_day == "evening":
            if self.went_to_social:
                self.move_to_residence()
                self.went_to_social = False
        
        self.decide_mask_wearing()
        
        self.interact_with_neighbors()

    def interact_with_neighbors(self):
        if not self.isolated:
            neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
            neighbors = [self.model.grid.get_cell_list_contents([node]) for node in neighbors_nodes]
            neighbors = [item for sublist in neighbors for item in sublist]

            for neighbor in neighbors:
                if isinstance(neighbor, CityAgent) and neighbor.health_status == "Infected":
                    transmission_chance = self.model.transmission_rate
                    if self.wearing_mask:
                        transmission_chance *= 0.5 
                    if random.random() < transmission_chance:
                        self.health_status = "Exposed"

# CityModel 类定义
class CityModel(Model):
    def __init__(self, num_agents, width, height, num_districts):
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.time_of_day = "morning"
        self.lockdown_areas = set()
        self.lockdown_coordinates = {}
        self.districts = self.create_districts(num_districts, width, height)
        
        # social places and perceived risk
        self.social_places = [random.choice(list(self.districts.keys())) for _ in range(5)]
        self.perceived_risk = perceived_risk
        self.transmission_rate = transmission_rate


        for i in range(self.num_agents):
            residence_area = (random.randint(0, width - 1), random.randint(0, height - 1))
            work_area = (random.randint(0, width - 1), random.randint(0, height - 1))
            district = self.districts[residence_area]
            agent = CityAgent(i, self, residence_area, work_area, district)
            self.schedule.add(agent)
            self.grid.place_agent(agent, residence_area)
        
        self.agent_movements = {i: [] for i in range(self.num_agents)}

    def create_districts(self, num_districts, width, height):
        districts = {}
        num_cells = width * height
        cells_per_district = num_cells // num_districts
        district_id = 0
        for x in range(width):
            for y in range(height):
                if len([v for v in districts.values() if v == district_id]) >= cells_per_district:
                    district_id += 1
                if district_id >= num_districts:
                    district_id = num_districts - 1
                districts[(x, y)] = district_id
        return districts

    def is_lockdown(self, area):
        return self.districts.get(area) in self.lockdown_areas

    def set_lockdown(self, district_id):
        self.lockdown_areas.add(district_id)
        self.lockdown_coordinates[district_id] = [coord for coord, dist in self.districts.items() if dist == district_id]

    def lift_lockdown(self, district_id):
        self.lockdown_areas.discard(district_id)
        if district_id in self.lockdown_coordinates:
            del self.lockdown_coordinates[district_id]

    def print_lockdown_areas(self):
        for district_id, coords in self.lockdown_coordinates.items():
            print(f"Lockdown District {district_id}: {coords}")

    def print_districts(self):
        district_coords = {}
        for coord, district_id in self.districts.items():
            if district_id not in district_coords:
                district_coords[district_id] = []
            district_coords[district_id].append(coord)
        
        for district_id, coords in district_coords.items():
            print(f"District {district_id}: {coords}")

    def update_perceived_risk(self):
        # calculate and update perceived risk based on infection
        total_agents = len(self.schedule.agents)
        infected_agents = sum(1 for agent in self.schedule.agents if agent.health_status == "Infected")
        self.perceived_risk = infected_agents / total_agents if total_agents > 0 else 0
        
    def step(self):
        self.schedule.step()
        for agent in self.schedule.agents:
            self.agent_movements[agent.unique_id].append(self.districts[agent.location])

        # add evening
        if self.time_of_day == "morning":
            self.time_of_day = "afternoon"
        elif self.time_of_day == "afternoon":
            self.time_of_day = "evening"
        else:
            self.time_of_day = "morning"

        
        # update perceived risk
        self.update_perceived_risk()
    

In [5]:
class SIERDAgent(Agent):
    def __init__(self, unique_id, model, wearing_mask=False, isolated=False, recovered=False):
        super().__init__(unique_id, model)
        self.state = "Susceptible"  # Initial state
        self.infection_time = 0
        self.wearing_mask = wearing_mask
        self.isolated = isolated
        self.recovered = recovered

    def step(self):
        if self.state == "Susceptible":
            self.check_exposure()
        elif self.state == "Exposed":
            self.progress_to_infected()
        elif self.state == "Infected":
            self.infect_others()
            self.progress_to_recovered_or_dead()
        elif self.state == "Recovered":
            self.check_reinfection()
        elif self.state == "Dead":
            pass  # No action

    def check_exposure(self):
        if self.isolated:
            return
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        for neighbor in neighbors:
            if neighbor.state == "Infected":
                transmission_rate = self.model.transmission_rate
                if self.wearing_mask:
                    transmission_rate *= 0.2
                if random.random() < transmission_rate:
                    self.state = "Exposed"
                    self.infection_time = self.model.schedule.time
                    break

    def progress_to_infected(self):
        if self.model.schedule.time - self.infection_time >= self.model.latency_period:
            self.state = "Infected"

    def infect_others(self):
        if self.isolated:
            return
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        for neighbor in neighbors:
            if neighbor.state == "Susceptible":
                transmission_rate = self.model.transmission_rate
                if self.wearing_mask:
                    transmission_rate *= 0.2
                if neighbor.recovered:
                    transmission_rate *= 0.5  # Lower rate for those who have recovered
                if random.random() < transmission_rate:
                    neighbor.state = "Exposed"
                    neighbor.infection_time = self.model.schedule.time

    def progress_to_recovered_or_dead(self):
        if self.model.schedule.time - self.infection_time >= self.model.infection_duration:
            if random.random() < self.model.recovery_rate:
                self.state = "Recovered"
                self.recovered = True
                if not self.wearing_mask:  # Recovered individuals are more likely to wear masks
                    self.wearing_mask = random.random() < 0.8
            else:
                self.state = "Dead"

    def check_reinfection(self):
        if self.isolated:
            return
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        for neighbor in neighbors:
            if neighbor.state == "Infected":
                transmission_rate = self.model.transmission_rate  # Same reinfection rate as initial infection
                if self.wearing_mask:
                    transmission_rate *= 0.2
                if random.random() < transmission_rate:
                    self.state = "Exposed"
                    self.infection_time = self.model.schedule.time
                    break

class SIERDModel(Model):
    def __init__(self, width, height, density, transmission_rate, latency_period, infection_duration, recovery_rate, policy):
        self.num_agents = int(width * height * density)
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.transmission_rate = transmission_rate
        self.latency_period = latency_period
        self.infection_duration = infection_duration
        self.recovery_rate = recovery_rate
        self.policy = policy

        for i in range(self.num_agents):
            wearing_mask = False
            isolated = False
            if policy == "Mask Policy Only":
                wearing_mask = True
            elif policy == "Lockdown Only":
                isolated = True
            elif policy == "Combination of Lockdown and Mask Policy":
                wearing_mask = True
                isolated = True
            
            agent = SIERDAgent(i, self, wearing_mask, isolated)
            self.schedule.add(agent)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

        # Initialize some agents as infected
        infected_agents = self.random.sample(self.schedule.agents, initial_infected)
        for agent in infected_agents:
            agent.state = "Infected"
            agent.infection_time = self.schedule.time

        self.datacollector = DataCollector(
            agent_reporters={"State": "state"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [6]:
# 参数设置
width = 50
height = 50
density = 0.1
transmission_rate = 0.2
latency_period = 3
infection_duration = 14
recovery_rate = 0.95
initial_infected = 10
social_places = [random.randint(0, 49) for _ in range(5)]  # 5 random social places
perceived_risk = 0.5  # initial perceived risk

# 定义政策
policies = ["No Interventions", "Lockdown Only", "Mask Policy Only", "Combination of Lockdown and Mask Policy"]

# 为每种政策运行模型
for policy in policies:
    print(f"Running model with policy: {policy}")
    model = SIERDModel(width, height, density, transmission_rate, latency_period, infection_duration, recovery_rate, policy)
    for i in range(100):
        model.step()
    
    # 数据分析
    data = model.datacollector.get_agent_vars_dataframe()
    print(data)

Running model with policy: No Interventions
                    State
Step AgentID             
0    0        Susceptible
     1        Susceptible
     2        Susceptible
     3        Susceptible
     4        Susceptible
...                   ...
99   93       Susceptible
     54       Susceptible
     109      Susceptible
     90       Susceptible
     81       Susceptible

[25000 rows x 1 columns]
Running model with policy: Lockdown Only


C:\Users\47324\AppData\Local\Temp\ipykernel_5016\1559170933.py:3: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)
C:\Users\47324\AppData\Local\Temp\ipykernel_5016\1559170933.py:3: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


                    State
Step AgentID             
0    0        Susceptible
     1        Susceptible
     2        Susceptible
     3        Susceptible
     4        Susceptible
...                   ...
99   62       Susceptible
     60       Susceptible
     129      Susceptible
     66       Susceptible
     43       Susceptible

[25000 rows x 1 columns]
Running model with policy: Mask Policy Only
                    State
Step AgentID             
0    0        Susceptible
     1        Susceptible
     2        Susceptible
     3        Susceptible
     4        Susceptible
...                   ...
99   241      Susceptible
     66       Susceptible
     75       Susceptible
     150      Susceptible
     237      Susceptible

[25000 rows x 1 columns]
Running model with policy: Combination of Lockdown and Mask Policy


C:\Users\47324\AppData\Local\Temp\ipykernel_5016\1559170933.py:3: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)
C:\Users\47324\AppData\Local\Temp\ipykernel_5016\1559170933.py:3: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


                    State
Step AgentID             
0    0        Susceptible
     1        Susceptible
     2        Susceptible
     3        Susceptible
     4        Susceptible
...                   ...
99   68       Susceptible
     134      Susceptible
     150      Susceptible
     231      Susceptible
     202      Susceptible

[25000 rows x 1 columns]
